In [6]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('data/WalkingKiwi_Tracks4.csv', index_col=0)
df.head()
df.shape

,name,type,time_h,length_km,minElevation,maxElevation,netElevation,totalAscent,lng,lat,coordinates,trackElevation,url,region
0,L. Wairoa Road End to Barrys Flat,Tramping Track,7.07,11.51,178.5,407.1,228.6,785.8,173.096578,-41.494357,"173.096578,-41.494357 173.096810,-41.494361 17...",192.003540039000000660962541587650775909423828...,-l-wairoa-road-end-to-barrys-flat,Tasman
1,Ngatuhoa Stream Track,Tramping Track,3.80,6.45,318.8,461.0,142.2,340.1,176.006116,-37.927966,"176.006116,-37.927966 176.006240,-37.927909 17...",325.382751464839998334355186671018600463867187...,-ngatuhoa-stream-track,Bay Of Plenty
2,(coastal) Smoothwater-staffordhut Route,Route,2.30,5.84,0.0,37.6,37.6,214.0,168.594405,-43.975153,"168.594405,-43.975153 168.594300,-43.975123 16...","1,0.317498326301569988583395343084703199565410...",-coastal-smoothwaterstaffordhut-route,West Coast
3,(inland) Smoothwater-staffordhut Route,Route,2.98,6.99,1.3,253.1,251.8,387.9,168.545144,-43.996206,"168.545144,-43.996206 168.545165,-43.996220 16...",2.98784279823299980805018094542901962995529174...,-inland-smoothwaterstaffordhut-route,West Coast
4,1000 Acres (1000 Ac Jcn to Pp Hut),Tramping Track,3.35,4.42,345.6,1099.0,753.4,679.6,172.331671,-41.649679,"172.331671,-41.649679 172.330260,-41.646983 17...","345.5771484375,476.229125976559998889570124447...",1000-acres-1000-ac-jcn-to-pp-hut,Tasman


(3075, 14)

In [7]:
trails = df[['name','type','time_h','length_km','netElevation','totalAscent','lng','lat','region']].copy(deep=True)
trails.head(2)

,name,type,time_h,length_km,netElevation,totalAscent,lng,lat,region
0,L. Wairoa Road End to Barrys Flat,Tramping Track,7.07,11.51,228.6,785.8,173.096578,-41.494357,Tasman
1,Ngatuhoa Stream Track,Tramping Track,3.80,6.45,142.2,340.1,176.006116,-37.927966,Bay Of Plenty


In [8]:
trails = pd.get_dummies(trails, columns=['type','region'])
trails.head(2)

,name,time_h,length_km,netElevation,totalAscent,lng,lat,type_Easy Tramping Track,type_Great Walk,type_Route,type_Short Walk,type_Short Walk(disabled),type_Track - historic,type_Tramping Track,type_Walking Track,region_Auckland,region_Bay Of Plenty,region_Canterbury,region_Chatham Islands,region_Gisborne,region_Hawkes Bay,region_ManawatuWanganui,region_Marlborough,region_Nelson,region_Northland,region_Otago,region_Southland,region_Taranaki,region_Tasman,region_Waikato,region_Wellington,region_West Coast
0,L. Wairoa Road End to Barrys Flat,7.07,11.51,228.6,785.8,173.096578,-41.494357,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Ngatuhoa Stream Track,3.80,6.45,142.2,340.1,176.006116,-37.927966,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Extract only some subset of columns to reduce computation time 
X = trails.drop(columns='name').values

# Standardize the features so that no feature dominates the distance computations due to unit scale
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [11]:
test = trails.drop(columns='name')[0:1].values
test = scaler.transform(test)

# Distance from all other cars
distances = euclidean_distances(X, test)
distances = distances.reshape(-1)   # Before it was (n_cars, 1)

# Find the 3 indices with the minimum distance (highest similarity) to the car we're looking at
ordered_indices = distances.argsort()
closest_indices = ordered_indices[1:6]

# Get the cars for these indices
closest_trails = trails.iloc[closest_indices]
closest_trails

,name,time_h,length_km,netElevation,totalAscent,lng,lat,type_Easy Tramping Track,type_Great Walk,type_Route,type_Short Walk,type_Short Walk(disabled),type_Track - historic,type_Tramping Track,type_Walking Track,region_Auckland,region_Bay Of Plenty,region_Canterbury,region_Chatham Islands,region_Gisborne,region_Hawkes Bay,region_ManawatuWanganui,region_Marlborough,region_Nelson,region_Northland,region_Otago,region_Southland,region_Taranaki,region_Tasman,region_Waikato,region_Wellington,region_West Coast
1475,Matiri Valley Track (mcc Hut to Hur Hut),7.43,12.37,378.1,752.0,172.364479,-41.587552,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1054,Karamea Bend Hut to Crow River,6.00,9.71,157.0,688.1,172.468717,-41.292658,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2659,Tiraumea Track - Tutaki Rd to T.hut,8.15,14.14,250.4,643.1,172.484376,-41.838739,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
628,End Grass Flats to Cecil Kings Hut,5.72,9.22,157.1,660.3,172.479318,-41.466902,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2217,Robert Ridge Route,5.92,9.20,380.2,788.2,172.748239,-41.888042,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
